In [1]:
from pydisagg.disaggregate import split_datapoint
import numpy as np
from pydisagg.models import RateMultiplicativeModel
from pydisagg.models import LMO_model
from pydisagg.models import LogOdds_model
import pandas as pd

### Basic example using RateMultiplicativeModel and OddsMultiplicativeModel

In [2]:
populations=np.array([20,10,5])
rate_pattern=np.array([0.1,0.5,0.7])
observed_total=20.
observed_total_SE=2.5

In [3]:
rmm=RateMultiplicativeModel()
oddm=LogOdds_model()

In [4]:
print(oddm.fit_beta(observed_total,rate_pattern,populations))
print(rmm.fit_beta(observed_total,rate_pattern,populations))

1.596597932398452
0.6443570163905132


In [5]:
print(rmm.split_to_rates(observed_total,rate_pattern,populations))
print(oddm.split_to_rates(observed_total,rate_pattern,populations))

[0.19047619 0.95238095 1.33333333]
[0.35420034 0.83154236 0.92011391]


In [6]:
b=rmm.fit_beta(observed_total,populations,rate_pattern)

bse=rmm.beta_standard_error(
    b,
    populations,
    rate_pattern,
    observed_total_SE
)
print(bse)
print(bse * rmm.count_diff_beta(b,rate_pattern,populations))
print(np.sum(bse * rmm.count_diff_beta(b,rate_pattern,populations)))

0.125
[0.47619048 1.19047619 0.83333333]
2.5


In [7]:
rmm.count_split_standard_errors(b,rate_pattern,populations,observed_total_SE)

array([0.47619048, 1.19047619, 0.83333333])

In [8]:
oddm.split_to_rates(observed_total,rate_pattern,populations)

array([0.35420034, 0.83154236, 0.92011391])

In [9]:
oddm.fit_beta(observed_total,rate_pattern,populations)

1.596597932398452

In [10]:
b=oddm.fit_beta(observed_total,rate_pattern,populations)

In [11]:
1/oddm.H_diff_beta(b,rate_pattern,populations)

0.15764994875370109

In [12]:
oddm.beta_standard_error(
    b,
    rate_pattern,
    populations,
    observed_total_SE
)

0.3941248718842527

In [13]:
pattern_cov=(np.ones((3,3))+np.identity(3)*2)*0.1

In [14]:
oddm.parameter_covariance(b,rate_pattern,populations,observed_total_SE,pattern_cov)

array([[21.58432567, -2.52000703, -1.0939538 , -0.97246609],
       [-2.52000703,  0.3       ,  0.1       ,  0.1       ],
       [-1.0939538 ,  0.1       ,  0.3       ,  0.1       ],
       [-0.97246609,  0.1       ,  0.1       ,  0.3       ]])

In [15]:
C=oddm.count_split_covariance_uncertainty(b,rate_pattern,populations,observed_total_SE,pattern_cov)
C

array([[ 54.85987474, -40.67511513,  -9.67710498],
       [-40.67511513,  34.59942445,   7.45591265],
       [ -9.67710498,   7.45591265,   2.58331574]])

In [16]:
pcov=oddm.parameter_covariance(b,rate_pattern,populations,observed_total_SE,pattern_cov)
count_jac = oddm.count_jacobian(b,rate_pattern,populations)
count_jac@pcov@count_jac.T

array([[ 54.85987474, -40.67511513,  -9.67710498],
       [-40.67511513,  34.59942445,   7.45591265],
       [ -9.67710498,   7.45591265,   2.58331574]])

In [17]:
count_jac

array([[ 4.57484919, 50.83165761,  0.        ,  0.        ],
       [ 1.40079662,  0.        ,  5.60318649,  0.        ],
       [ 0.36752149,  0.        ,  0.        ,  1.75010235]])

In [18]:
pcov

array([[21.58432567, -2.52000703, -1.0939538 , -0.97246609],
       [-2.52000703,  0.3       ,  0.1       ,  0.1       ],
       [-1.0939538 ,  0.1       ,  0.3       ,  0.1       ],
       [-0.97246609,  0.1       ,  0.1       ,  0.3       ]])

In [19]:
h=0.001
(oddm.predict_count(
    b,
    rate_pattern+h*np.array([0,1,0]),
    populations
)-oddm.predict_count(
    b,
    rate_pattern-h*np.array([0,1,0]),
    populations
))/(2*h)

array([0.        , 5.60319634, 0.        ])

In [20]:
h=0.001
(oddm.split_to_counts(
    observed_total,
    rate_pattern+h*np.array([0,1,0]),
    populations
)-oddm.split_to_counts(
    observed_total,
    rate_pattern-h*np.array([0,1,0]),
    populations
))/(2*h)

array([-4.04116183,  4.36580924, -0.32464741])

In [21]:
jfit=oddm.parameter_fit_jacobian(b,rate_pattern,populations)

In [22]:
pcov

array([[21.58432567, -2.52000703, -1.0939538 , -0.97246609],
       [-2.52000703,  0.3       ,  0.1       ,  0.1       ],
       [-1.0939538 ,  0.1       ,  0.3       ,  0.1       ],
       [-0.97246609,  0.1       ,  0.1       ,  0.3       ]])

In [23]:
datacov=np.block(
    [
        [np.array([[observed_total_SE**2]]),np.zeros((1,3))],
        [np.zeros((3,1)),pattern_cov]
    ]
)

In [24]:
jfit@datacov@jfit.T

array([[21.58432567, -2.52000703, -1.0939538 , -0.97246609],
       [-2.52000703,  0.3       ,  0.1       ,  0.1       ],
       [-1.0939538 ,  0.1       ,  0.3       ,  0.1       ],
       [-0.97246609,  0.1       ,  0.1       ,  0.3       ]])

In [25]:
jfit

array([[ 0.15764995, -8.01360822, -0.88334206, -0.27590355],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [26]:
rj = oddm.rate_jacobian(b,rate_pattern)

In [27]:
h=0.01
(oddm.fit_beta(
    observed_total,
    rate_pattern+h*np.array([1,0,0]),
    populations
)-oddm.fit_beta(
    observed_total,
    rate_pattern-h*np.array([1,0,0]),
    populations
))/(2*h)

-8.042993289678002

In [28]:
jfit

array([[ 0.15764995, -8.01360822, -0.88334206, -0.27590355],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  1.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])